# Transfer Learning

### 1. Utilizar un modelo pre-entrenado:

In [14]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

image = load_img('./8-CNN/perro.jpg', target_size=(224, 224))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# preparamos la imagen para el modelo y cargamos el modelo
image = preprocess_input(image)
model = VGG16(weights='imagenet', include_top=True)
# predicción
yhat = model.predict(image)           # Probabilidades
label = decode_predictions(yhat)      # Etiquetas
label = label[0][0]

print('%s (%.2f%%)' % (label[1], label[2]*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
Walker_hound (50.14%)


### 2. Feature transfer:

In [17]:
import keras as tf
from keras.datasets import cifar10
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
import matplotlib.pyplot as plt

# Load CIFAR-10 dataset
(train_data, train_labels), (val_data, val_labels) = cifar10.load_data()

# Normalize pixel values to [0, 1]
train_data = train_data / 255.0
val_data = val_data / 255.0

# One-hot encode labels
train_labels = to_categorical(train_labels, num_classes=10)
val_labels = to_categorical(val_labels, num_classes=10)

In [18]:
# Build the model using VGG16 as base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
pretrain_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the pretrained model
for layer in base_model.layers:
    layer.trainable = False

# Compile the pretraining model
pretrain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [19]:
# Fit the pretraining model
history_pretrain = pretrain_model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))


Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 303s 194ms/step - accuracy: 0.4865 - loss: 1.4578 - val_accuracy: 0.5670 - val_loss: 1.2205
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 303s 193ms/step - accuracy: 0.5940 - loss: 1.1451 - val_accuracy: 0.5913 - val_loss: 1.1694
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 298s 191ms/step - accuracy: 0.6269 - loss: 1.0653 - val_accuracy: 0.5937 - val_loss: 1.1521
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 296s 189ms/step - accuracy: 0.6423 - loss: 1.0054 - val_accuracy: 0.6024 - val_loss: 1.1495
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 298s 190ms/step - accuracy: 0.6677 - loss: 0.9391 - val_accuracy: 0.6152 - val_loss: 1.1187
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 296s 189ms/step - accuracy: 0.6881 - loss: 0.8871 - val_accuracy: 0.6193 - val_loss: 1.0932
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 298s 191ms/step - accuracy: 0.7006 - loss: 0.8410 - val_accuracy: 0.6233 - val_loss: 1.1186
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 299s 191ms/step - ac

### 3. Fine tunning:

In [22]:
# Build the model using VGG16 as base
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
pretrain_model = Model(inputs=base_model.input, outputs=predictions)
pretrain_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# Fit the pretraining model
history_pretrain = pretrain_model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

# Freeze the layers of the pretrained model
for layer in base_model.layers:
    layer.trainable = False

# Fine-tuning the model
for layer in pretrain_model.layers[-4:]:
    layer.trainable = True

# Compile the model with a lower learning rate
pretrain_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the fine-tuning model
history_finetune = pretrain_model.fit(train_data, train_labels, epochs=5, validation_data=(val_data, val_labels))



Epoch 1/10
  65/1563 ━━━━━━━━━━━━━━━━━━━━ 13:30 541ms/step - accuracy: 0.0966 - loss: 2.4289

KeyboardInterrupt: 

In [ ]:
# Evaluate the model on validation data
val_loss, val_accuracy = pretrain_model.evaluate(val_data, val_labels, verbose=2)
print(f'Validation Accuracy: {val_accuracy:.4f}')
print(f'Validation Loss: {val_loss:.4f}')

# Plot training & validation accuracy values
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history_pretrain.history['accuracy'] + history_finetune.history['accuracy'])
plt.plot(history_pretrain.history['val_accuracy'] + history_finetune.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history_pretrain.history['loss'] + history_finetune.history['loss'])
plt.plot(history_pretrain.history['val_loss'] + history_finetune.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()
